# Домашнее задание №2

Это задание состоит из двух сюжетов:

1) Workflow ML-модели. Несложный код, который вы пишете всегда - чтобы обучить модель от начала (загрузка данных) и до конца (оценка качества и улучшение модели).

2) Самостоятельная реализация различных способов кодирования категориальных признаков - полезное упражнение для понимания кодировок (в дальнейшем будем использовать готовые функции).

## <font color='green'>Часть 1. ML workflow (**всего 5 баллов**)</font>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы. 

In [2]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)
print(data.DESCR)
df = data.frame
df.head()

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


Будем решать задачу регрессии: необходимо предсказать уровень сахара в крови по характеристикам пациентов

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("target")

In [3]:
X = np.array(df[df.columns[:10]])
#print(X)
y = np.array(df['target'])
y

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
        68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
        87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
       259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
       128.,  52.,  37., 170., 170.,  61., 144.,  52., 128.,  71., 163.,
       150.,  97., 160., 178.,  48., 270., 202., 111.,  85.,  42., 170.,
       200., 252., 113., 143.,  51.,  52., 210.,  65., 141.,  55., 134.,
        42., 111.,  98., 164.,  48.,  96.,  90., 162., 150., 279.,  92.,
        83., 128., 102., 302., 198.,  95.,  53., 134., 144., 232.,  81.,
       104.,  59., 246., 297., 258., 229., 275., 281., 179., 200., 200.,
       173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
       107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,
        60., 174., 259., 178., 128.,  96., 126., 28

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
#print(len(X_train))
#print(len(y_train))

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [5]:
lr = LinearRegression().fit(X_train, y_train)
y_pred_train = lr.predict(X_train)
#print(y_pred_train)

y_pred_test = lr.predict(X_test)
#print(y_pred_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [6]:
print('MSE на train:', mean_squared_error(y_pred_train, y_train))
print('MSE на test:', mean_squared_error(y_pred_test, y_test))

print('r2 на train:', r2_score(y_pred_train, y_train))
print('r2 на test:', r2_score(y_pred_test, y_test))




MSE на train: 2696.1673856701314
MSE на test: 3390.7470812208194
r2 на train: 0.15357417933833406
r2 на test: -0.4105821041163882


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

Вообще непонятно из условия, где мы делаем кросс-валидацию. Буду считать, что на всем X

In [7]:
print(cross_val_score(lr, X, y, cv=5, scoring = "r2"))

[0.42955615 0.52259939 0.48268054 0.42649776 0.55024834]


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

Снова расплывчатое условие, непонятно, где предсказываем нашу модель (которую обучили на тренировочных данных). Буду считать, что на train. Также непонятно, по какой метрике мы оцениваем качество, буду считать, что по MSE

In [8]:
from sklearn.linear_model import Lasso

tmp = []
tmp = []
for alpha in range(1, 11, 1):
  #print(alpha)
  y_pred_lasso_i = Lasso(alpha=alpha/10).fit(X_train, y_train).predict(X_train)
  tmp.append(mean_squared_error(y_pred_lasso_i, y_train))
print(tmp)
print(min(tmp))


[2742.519979772501, 2832.463527911427, 2935.5749401615285, 3049.59173276185, 3124.0021929165227, 3214.3295812099427, 3321.0798822423467, 3425.1023306363877, 3524.6677851163568, 3635.9358068101014]
2742.519979772501


Получили min MSE при α = 0.1

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

Я не понимаю, что конкретно нужно в шагах 6-8. Качество и лучшую модель я вывел в шаге 6 (α = 0.1 и MSE = 2884.8). ЧТо здесь нужно я не знаю. Вывел коэффициенты

In [9]:
lr_lasso = Lasso(alpha=0.1).fit(X_train, y_train)
y_pred_lasso = lr_lasso.predict(X_train)
print(lr_lasso.coef_) 
y_pred_lasso_test = lr_lasso.predict(X_test)


[  -0.         -206.5880884   493.31168597  197.74842819   -0.
  -79.27849028 -265.37959469    0.          488.03560256   78.46327945]


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [10]:
print('r2 на test:', r2_score(y_pred_lasso_test, y_test))
print(y_pred_lasso_test)

r2 на test: -0.5463789558026406
[209.39548292 217.8590901  134.18462777 185.105315   163.12970989
  99.82550798 106.30169934  96.01816495 208.75425585 136.46933818
 123.35602795 168.81701189 180.38940325 102.61120155 165.56519148
 154.04176725 195.2129783  153.60379634 223.62462609 160.55396053
  84.2744278  108.44227579 152.71914332  91.77138888 190.43975518
 241.99188881 101.53938045 221.64941466 197.91750174 171.19144091
 105.54733367 274.18794649  86.36297327 155.14193156 204.27486139
 260.11326591 126.82388062 161.52203892 101.40550294 162.99108282
  81.12785931 202.43110273  90.99057599 199.92157311 165.80721823
 190.52415691 232.24337104 168.74200267 132.82825351 150.84904409
 171.50652088 203.84309055 124.80006857 155.86625277 191.9590511
 188.76567683 164.54674958  68.40277228 130.40049466 158.49020029
  62.84072282  70.50132656 130.86489271 252.56172503 107.88121994
 172.03652677 164.8393639  157.01405819 167.83999845 127.9716831
 215.23875455 142.16074236  85.45184917 196.79

### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [11]:
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([('scaling', StandardScaler()), ('pol', PolynomialFeatures()), ('regr',LinearRegression())])
print(cross_val_score(pipeline, X, y, cv=5, scoring = "r2"))

[0.36658098 0.50146703 0.44620279 0.16882057 0.47951987]


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [12]:
from sklearn.preprocessing import StandardScaler

pipeline_new = Pipeline([('scaling', StandardScaler()), ('pol', PolynomialFeatures()), ('regr',LinearRegression())])
pipeline_new.fit(X_train, y_train)
y_pred_train_poly = pipeline_new.predict(X_train)
y_pred_test_poly= pipeline_new.predict(X_test)


print('MSE на train:', mean_squared_error(y_pred_train_poly, y_train))
print('MSE на test:',  mean_squared_error(y_pred_test_poly, y_test))


print('r2 на train:', r2_score(y_pred_train_poly, y_train))
print('r2 на test:', r2_score(y_pred_test_poly, y_test))

print(y_pred_train_poly)
print(y_pred_test_poly)


MSE на train: 2049.677496690966
MSE на test: 6883.668015931782
r2 на train: 0.4651700803108233
r2 на test: -0.18910774615394366
[ 84.51367188  81.8046875  265.0703125  109.234375   141.7109375
 153.65234375 118.34375    117.15234375  81.38671875 243.453125
  63.09375     87.86328125 271.94140625 113.35742188 181.12695312
 227.88671875 136.5        182.34765625 112.50585938 132.79296875
  55.35742188 152.52148438 151.76757812 115.31054688 144.4296875
 141.7265625  220.27148438 256.34570312  63.63671875 112.41601562
 102.65625    129.578125   192.1484375  124.859375    74.390625
 117.62890625 153.91210938 162.61523438  71.58203125 116.75585938
 254.4296875  176.75195312 206.25       104.32226562 181.35546875
 100.7734375  176.109375   129.03125     62.90820312 115.38867188
 159.19726562  79.328125    62.66210938 163.47070312  99.63671875
  92.91210938 204.34179688 119.859375   206.59375     58.59375
 202.33203125 264.08398438  98.34765625 108.3828125  123.94726562
 135.00390625 209.65234

### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

1) Плохо себя показала. MSE и r2 существенно хуже на тесте, чем на трэйне. Да, модель переобучена, r2<0 - это плохо (на трэйне)
2) Нет, даже скорее хуже сделала. r2 все еще плох
3) Полиномиальные признаки с кросс-валидацией ведут себя почти так же, как просто кросс-валидация, даже немного хуже. Но по сранению с L1 и просто трэйн и тест - стало лучше (см r2). Но поскольку мы добавили кучу признаков, то скорее всего мы переобучились (а если еще посмотреть на структуру данных - там столбцы похоже, что коррелируют друг с другом, а когда добавляет новые признаки - мы начинаем переобучаться. Идейно как можно поправить модель (след. задание) - это посмотреть на корреляции и выкинуть лишнее, после этого кросс-валидацию сделать)

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы 
(**дополнительно 1 балл**)

Немного идейно рассказал в прошлом задании, как делать. Если что-то дадите, то хорошо

## <font color='green'>Часть 2. Target encoding (**всего 5 баллов**)</font>

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [19]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [21]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
all_data

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0
...,...,...,...,...
10768834,59,22162,33,0.0
10769024,59,22163,33,0.0
10769690,59,22164,33,0.0
10771216,59,22166,33,0.0


### Mean encodings без регуляризации




После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [23]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

print(all_data)
# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

          shop_id  item_id  date_block_num  target  item_target_enc
139255          0       19               0     0.0         0.022222
141495          0       27               0     0.0         0.056834
144968          0       28               0     0.0         0.141176
142661          0       29               0     0.0         0.037383
138947          0       32               0     6.0         1.319042
...           ...      ...             ...     ...              ...
10768834       59    22162              33     0.0         1.556793
10769024       59    22163              33     0.0         0.581395
10769690       59    22164              33     0.0         1.235589
10771216       59    22166              33     0.0         0.295918
10770511       59    22167              33     0.0         1.081081

[10913850 rows x 5 columns]
0.4830386988621699


#### Способ 2

In [25]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

print(all_data)
# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

          shop_id  item_id  date_block_num  target  item_target_enc
139255          0       19               0     0.0         0.022222
141495          0       27               0     0.0         0.056834
144968          0       28               0     0.0         0.141176
142661          0       29               0     0.0         0.037383
138947          0       32               0     6.0         1.319042
...           ...      ...             ...     ...              ...
10768834       59    22162              33     0.0         1.556793
10769024       59    22163              33     0.0         0.581395
10769690       59    22164              33     0.0         1.235589
10771216       59    22166              33     0.0         0.295918
10770511       59    22167              33     0.0         1.081081

[10913850 rows x 5 columns]
0.4830386988621699


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

globalmean взял из следующего задания

In [40]:
from sklearn.model_selection import KFold

globalmean = 0.3343
KF = KFold(n_splits=5)

all_data["item_target_enc"] = all_data["target"]

for i, j in enumerate(KF.split(all_data)):
  train, test = j[0], j[1]
  tmp = (all_data.loc[train, :]).groupby("item_id").target.mean()
  all_data.loc[test, "item_target_enc"] = all_data.loc[test, :]["item_id"].map(tmp)

all_data['item_target_enc'].fillna(globalmean, inplace=True) 

encoded_feature = all_data["item_target_enc"].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(all_data)

0.4166330182247348
          shop_id  item_id  date_block_num  target  item_target_enc
139255          0       19               0     0.0         0.334300
141495          0       27               0     0.0         0.048421
144968          0       28               0     0.0         0.148485
142661          0       29               0     0.0         0.030303
138947          0       32               0     6.0         0.897048
...           ...      ...             ...     ...              ...
10768834       59    22162              33     0.0         0.020000
10769024       59    22163              33     0.0         0.334300
10769690       59    22164              33     0.0         0.334300
10771216       59    22166              33     0.0         0.349227
10770511       59    22167              33     0.0         1.229927

[10913850 rows x 5 columns]


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [41]:
target_sum = all_data.groupby("item_id").target.sum()
target_amount = all_data.groupby("item_id").target.size()
number_objects = all_data["item_id"].map(target_amount)

all_data["item_target_enc"] = (all_data["item_id"].map(target_sum) - all_data["target"])/(number_objects-1)
all_data['item_target_enc'].fillna(globalmean, inplace=True)


encoded_feature = all_data["item_target_enc"].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(all_data)

0.4803848311293002
          shop_id  item_id  date_block_num  target  item_target_enc
139255          0       19               0     0.0         0.022727
141495          0       27               0     0.0         0.056911
144968          0       28               0     0.0         0.141414
142661          0       29               0     0.0         0.037500
138947          0       32               0     6.0         1.316088
...           ...      ...             ...     ...              ...
10768834       59    22162              33     0.0         1.560268
10769024       59    22163              33     0.0         0.585938
10769690       59    22164              33     0.0         1.238693
10771216       59    22166              33     0.0         0.296170
10770511       59    22167              33     0.0         1.081967

[10913850 rows x 5 columns]


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [42]:
alpha = 100

target_mean = all_data.groupby("item_id").target.mean()
target_amount = all_data.groupby("item_id").target.size()

nrows = all_data["item_id"].map(target_amount)
all_data["item_target_enc"] = (all_data["item_id"].map(target_mean)*nrows + globalmean*alpha)/(nrows + alpha)
all_data['item_target_enc'].fillna(globalmean, inplace=True) 


encoded_feature = all_data["item_target_enc"].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(all_data)

0.4818198797097264
          shop_id  item_id  date_block_num  target  item_target_enc
139255          0       19               0     0.0         0.237448
141495          0       27               0     0.0         0.089905
144968          0       28               0     0.0         0.168964
142661          0       29               0     0.0         0.107910
138947          0       32               0     6.0         1.260635
...           ...      ...             ...     ...              ...
10768834       59    22162              33     0.0         1.334117
10769024       59    22163              33     0.0         0.473493
10769690       59    22164              33     0.0         1.054970
10771216       59    22166              33     0.0         0.298926
10770511       59    22167              33     0.0         1.024550

[10913850 rows x 5 columns]


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [43]:
target_previous_sum = all_data.groupby('item_id').target.cumsum() - all_data['target']
target_previous_amount = all_data.groupby('item_id').cumcount()

all_data['item_target_enc'] = target_previous_sum/target_previous_amount
all_data['item_target_enc'].fillna(globalmean, inplace=True) 


encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(all_data)

0.5025245211081697
          shop_id  item_id  date_block_num  target  item_target_enc
139255          0       19               0     0.0         0.334300
141495          0       27               0     0.0         0.334300
144968          0       28               0     0.0         0.334300
142661          0       29               0     0.0         0.334300
138947          0       32               0     6.0         0.334300
...           ...      ...             ...     ...              ...
10768834       59    22162              33     0.0         1.560268
10769024       59    22163              33     0.0         0.585938
10769690       59    22164              33     0.0         1.238693
10771216       59    22166              33     0.0         0.296170
10770511       59    22167              33     0.0         1.081967

[10913850 rows x 5 columns]


Ожидаемый ответ 0.5025